In [ ]:
from IPython.core.display import HTML
with open('style.css') as file:
    css = file.read()
HTML(css)

In [ ]:
# Autload python modules by default
%load_ext autoreload
%autoreload 2

# Convert notebooks to python, so they can be loaded effiently
from utils.jupyter_loader import JupyterLoader

loader = JupyterLoader()
loader.load_all()

# Quiescence Search

A current problem with the engine is that it always stops analyzing at a fixed depth, regardless of the position.
This can lead, for example, to the engine terminating the search in an exchange of blows and thus interpreting the position incorrectly.
Or the engine tries to extend a situation that is unfavorable for it, e.g. by giving check, so that this situation can no longer occur in the limited search depth.

This is also referred to as the [Horizon Effect](https://www.chessprogramming.org/Horizon_Effect).
To avoid this, the idea is to analyze only quiet positions with the heuristic. 
Quiet positions are those in which no piece can be captured and, depending on the implementation, no check can be given. 
In the other cases, a restricted search is made that analyzes precisely such moves even further. 
This is called [quiescence search](https://www.chessprogramming.org/Quiescence_Search).
A new class `QuiescenceEngine` is now added, which extends the previous engine `SimplifiedIncrementalEvaluationEngine`.

In [ ]:
from converted_notebooks.s12_simplified_evaluation import IncrementalIterativeAlphaBetaCached


class QuiescenceEngine(IncrementalIterativeAlphaBetaCached):
    pass

The `_value` function only needs a slight alteration: 
When the search depth is exhausted, the function `_quiescene` is called.

In [ ]:
import chess
from chess.engine import Score
from converted_notebooks.s11_iterative_deepening import cache_alpha_beta
from converted_notebooks.s12_simplified_evaluation import DetailedMove
from chess.engine import Score, PovScore


@cache_alpha_beta
def _value(
    self, board: chess.Board, depth: int, alpha: Score, beta: Score
) -> PovScore:
    if (score := self.evaluator.evaluate(board)) is not None:
        return score
    if depth == 0:
        return self._quiescence(board, alpha, beta)

    for move in board.legal_moves:
        detailedMove = DetailedMove(board, move)

        self.evaluator.push(detailedMove)
        board.push(move)
        value = self._value(board, depth - 1, -beta, -alpha).pov(not board.turn)
        board.pop()
        self.evaluator.pop()

        if value >= beta:
            return PovScore(value, board.turn)
        alpha = max(alpha, value)

    return PovScore(alpha, board.turn)


QuiescenceEngine._value = _value

The `_quiescene` method takes a chess board `board`, integers `alpha` and `beta` and returns a score of the current position similarly to the `_value` function.

Initially a static evaluation `stand_pat` of the current position, called a Standing Pat, is calculated.
If it is bigger than the upper limit `beta`, `beta` can be returned,
if it is bigger than the lower limit `alpha`, `alpha` needs to be raised to this value.


The logic itself is very similar to the logic of the `_value` function.
The first deviation is the restriction on capturing moves.
The second deviation is the use of optimization checks.
They are needed, as trading blows is possible in almost any position, leading to a massive increase of game tree.

Moves will be analyzed, if they are promoting a piece.
If they are not promoting a piece,
they may be pruned by DeltaPruning or a negative Static Exchange Evaluation.
These optimizations will be explained afterwards.
After those checks the usual implementation of NegaMax follows.

In [ ]:
from chess.engine import Score


def _quiescence(
    self, board: chess.Board, alpha: Score, beta: Score
) -> PovScore:
    stand_pat = self.evaluator.get_score().relative

    if stand_pat >= beta:
        return PovScore(beta, board.turn)
    alpha = max(alpha, stand_pat)

    for move in board.generate_legal_captures():
        detailed_move = DetailedMove(board, move)

        if move.promotion is None:
            if self._can_delta_prune(stand_pat, alpha, detailed_move):
                continue
            if self._see_capture(board, detailed_move) < 0:
                continue

        board.push(move)
        self.evaluator.push(detailed_move)

        value = self._quiescence(board, -beta, -alpha).pov(not board.turn)

        self.evaluator.pop()
        board.pop()

        if value >= beta:
            return PovScore(value, board.turn)
        alpha = max(alpha, value)
    return PovScore(alpha, board.turn)


QuiescenceEngine._quiescence = _quiescence

There are positions in which the player is so far behind, not even the best move can save them.
For example with one Queen behind, capturing a minor piece will not tip the scales.
Removing those moves is the idea of Delta Pruning. 

The method `_can_delta_prune` takes the Standing Pat `stand_pat`, the lower limit `alpha` and the detailed move `detailed_move` as parameters and returns a boolean indicating whether the move can be pruned.

Using the Standing Pat and the value of the potentially captured piece 
and a potential advantage of position of 2 Pawns = 200 Centipawns
the best case estimate is done for the reached position.
The advantage of position is very optimistic
as is the calculation of the opponent not capturing in return.

Is this best case estimate still smaller than `alpha`
it can be pruned,
so the method will return `True`.
The return value will be `False` otherwise.

In [ ]:
from chess.engine import Score, Cp


def _can_delta_prune(
    self, stand_pat: Score, alpha: Score, detailed_move: DetailedMove
) -> bool:
    POTENTIAL_POSITION_ADVANTAGE = 200
    pieceValue = self.evaluator.piece_values[
        detailed_move.capturedPiece.piece.piece_type]
    bestAlpha = Cp(
        stand_pat.score() + pieceValue + POTENTIAL_POSITION_ADVANTAGE
    )
    return bestAlpha < alpha


QuiescenceEngine._can_delta_prune = _can_delta_prune

The second optimization `Static Exchange Evaluation`
analyzes the potential slugfest.
It is expected for each player to capture using the lowest valued piece.
The exchange is only realized if the player has no loss from it.

The method `_see_capture` takes a chess board `board` and a detailed move `detailed_move` 
to return a score for the move.
If the score is positive, a gain is expected.
If it is negative, a loss is expected and the move therefore pruned in the `_quiescence` method.

The method tries the move, calculates the score using Static Exchange Evaluation, undoes the move and returns the score.
The calculation of the score is done by subtracting the value returned by the recursive `_see` method from the value of the captured piece.

In [ ]:
def _see_capture(self, board: chess.Board, detailed_move: DetailedMove) -> int:
    board.push(detailed_move.move)
    captured_piece = detailed_move.capturedPiece
    value = self.evaluator.piece_values[
        captured_piece.piece.piece_type
    ] - self._see(board, detailed_move.placedPiece.square)
    board.pop()
    return value


QuiescenceEngine._see_capture = _see_capture

The `_see` method uses the chess board `board` and the square `square` of the current exchange as parameters and returns the Static Exchange Evaluation score. 
Note that the square is taken from `detailedMove.placedPiece`, which handles edge cases such as en passant.
As the piece was already removed, its value is used in the calling `_see_capture` method.
The squares of all possible attackers of the exchange square 
are returned by the `attackers` function.
As pinned pieces can not participate in the exchange, 
they are identified using the `is_pinned` method and ignored.
The squares of the remaining attackers are stored in `attackers_squares`.
If there are no attackers left, the exchange is finished and 0 is returned.
Otherwise, the lowest valued piece is designated to be the new attacker 
and the move is evaluated
by subtracting the value of a recursive call to `_see` from the value of the captured piece.
As each player will only execute a capture, 
if there is no loss, 
the maximum of exchange value and 0 is returned,
after undoing the move.

In [ ]:
def _see(self, board: chess.Board, square: chess.Square) -> int:
    attackers_squares = [
        square for square in board.attackers(board.turn, square)
        if not board.is_pinned(board.turn, square)
    ]
    if not attackers_squares:
        return 0

    attacker_square = min(
        attackers_squares,
        key=lambda attackers_square: self.evaluator.piece_values[
            board.piece_type_at(attackers_square)]
    )
    captured_piece = board.piece_type_at(square)

    detailed_move = DetailedMove(
        board, chess.Move(from_square=attacker_square, to_square=square)
    )
    board.push(detailed_move.move)
    value = max(
        0,
        self.evaluator.piece_values[captured_piece] -
        self._see(board, detailed_move.placedPiece.square)
    )
    board.pop()
    return value


QuiescenceEngine._see = _see

To test the function a scenario with one pinned attacker is created and evaluated.

In [ ]:
import IPython.display
from converted_notebooks.s12_simplified_evaluation import incremental_simplified_evaluator

board = chess.Board("6k1/6b1/8/4p3/3P4/8/8/1K4R1 w - - 0 1")
IPython.display.display(board)
engine = QuiescenceEngine(
    evaluator=incremental_simplified_evaluator, max_look_ahead_depth=4
)
result = engine._see(board, chess.E5)
assert result == 100

As not all exchanges result in a net win, they should not be taken into consideration in the second test case.

In [ ]:
board = chess.Board("6k1/6b1/5p2/4q3/3P4/8/8/1K2Q3 w - - 0 1")
IPython.display.display(board)
engine = QuiescenceEngine(
    evaluator=incremental_simplified_evaluator, max_look_ahead_depth=4
)
result = engine._see(board, chess.E5)
assert result == 800

This newly defined engine can now be tested on the default `middlegame_board`.

In [ ]:
import random
from converted_notebooks.s09_minimax_engine import middlegame_board

random.seed(42)

engine = QuiescenceEngine(
    evaluator=incremental_simplified_evaluator, max_look_ahead_depth=4
)
engine.analyse(middlegame_board)

With the line profiler the `_quiescence` can be examined. It can be seen that the static exchange evaluation takes up the most time of the function, but on the other hand allows us to skip many moves and thus recursive calls.

In [ ]:
%load_ext line_profiler

In [ ]:
%lprun -f QuiescenceEngine._quiescence engine.analyse(middlegame_board)